# DBSCAN python sklearn performance

In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
#X, y = make_blobs(n_samples=1_500, n_features=2, cluster_std=.5, center_box=(- 10.0, 10.0), shuffle=True, random_state=None)
#print(X.shape)

X = np.array(range(2*1500), dtype=np.float32)

y = [0 for _ in range(1500)]

for i, x in enumerate(X):
  X[i] = (x % 20) - 10

X = np.reshape(X, (1500, 2), order='F')

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN
import multiprocessing

t0 = time.perf_counter()
clustering = DBSCAN(eps=.5, min_samples=5, n_jobs=-1).fit(X)
python_time = time.perf_counter() - t0
print(f'Elapsed time {python_time}s with {multiprocessing.cpu_count()} cpu')
clustering.labels_

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=clustering.labels_)
plt.show()

In [ ]:
from sklearn import metrics

metrics.adjusted_rand_score(y, clustering.labels_)

# Test and profiling CUDA Implementation

In [ ]:
!nvidia-smi

In [ ]:
%cd /content/drive/MyDrive/G-DBSCAN
#!nvcc --generate-line-info -arch=sm_75 gdbscan.cu && ./a.out
#k80
!nvcc --generate-line-info -arch=sm_37 gdbscan.cu && ./a.out

In [ ]:
!cuda-memcheck ./a.out | head -n 100

In [ ]:
!rm *.qdrep
!nsys profile ./a.out

In [ ]:
#!ncu --kernel-name compute_degrees --launch-skip 0 --launch-count 1 "/content/drive/My Drive/G-DBSCAN/./a.out" 
!ncu --kernel-name compute_degrees "/content/drive/My Drive/G-DBSCAN/./a.out" 

In [ ]:
!ncu --kernel-name compute_adjacency_list "/content/drive/My Drive/G-DBSCAN/./a.out" 